# **Изучение влияния семейного положения и количества детей на погашение кредита в срок.**

#### Заказчик — кредитный отдел банка. 
#### Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
#### Входные данные от банка — статистика о платёжеспособности клиентов.
#### Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# 1. Открытие и изучение БД

### Описание данных
`children` — количество детей в семье

`days_employed` — общий трудовой стаж в днях

`dob_years` — возраст клиента в годах

`education` — уровень образования клиента

`education_id` — идентификатор уровня образования

`family_status` — семейное положение

`family_status_id` — идентификатор семейного положения

`gender` — пол клиента

`income_type` — тип занятости

`debt` — имел ли задолженность по возврату кредитов

`total_income` — ежемесячный доход

`purpose` — цель получения кредита

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np

In [2]:
# Чтение БД в датафрейм
df = pd.read_csv('C:\\Users\\kovil\\Desktop\\Analitics\\data.csv')
# Общая информация о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


##### Из общей информации датафрейма мы видим:
* Всего 21525 строк, 12 колонок
* Нулевые значения есть только в столбцах `'days_employed'` и `'total_income'`.

In [3]:
# Просмотр первых 10 строк для общего ознакомления с данными
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


##### Из первых 10 строк видно:
* Некоторые значения в столбце `'days_employed'` имеют знак `'минус'`. **Понять, почему**
* Значения в столбце `'education'` бывают в разном регистре. **При работе с этим столбцом использовать lower() для приведения значений в нижний регистр**

# 2. Предобработка данных

### Приведение датафрейма в порядок

#### Пропуски

In [4]:
# Количество нулевых значений в 'total_income'
count_null_total_income = df['total_income'].isna().sum()
proportion_null_total_income = (count_null_total_income / 21525)
print(f'Нулевых значений в \'total_income\': {count_null_total_income}')
print(f'Доля нулевых значений в \'total_income\': {proportion_null_total_income:.1%}.')

# Количество нулевых значений в 'days_employed'
count_null_days_employed = df['days_employed'].isna().sum()
proportion_null_days_employed = (count_null_days_employed / 21525)
print(f'Нулевых значений в \'days_employed\': {count_null_days_employed}')
print(f'Доля нулевых значений в \'days_employed\': {proportion_null_days_employed:.1%}.')



Нулевых значений в 'total_income': 2174
Доля нулевых значений в 'total_income': 10.1%.
Нулевых значений в 'days_employed': 2174
Доля нулевых значений в 'days_employed': 10.1%.


В столбцах `'total_income'` и `'days_employed'` обнаружены пропущенные значения.
В каждом из них одинаковое количество пропущенных значений.
Пропущенные значения составляют **10%** от всех значений в каждом столбце.

**`Вероятно пропущенные значения находятся "в паре".`**

In [16]:
#Проверка "парности" пропущенных значений.
#null_total_income = df[df['total_income'].isna() == True]
#display(null_total_income)
null_total_income_and_employed = df.loc[(df['total_income'] == NaN)&(df['days_employed'] == NaN)].count()
print(null_total_income_and_employed)

NameError: name 'NaN' is not defined

Из таблицы видно, что пропущенные значения находятся **`"в паре"`**.
Можно предположить, что клиент не работает официально (не может предоставить документы о среднем заработке) или является частным предпринимателем. 

В столбце `'total_income'` есть нулевые значения.
Т.к. этот столбец отвечает за ежемесячный доход, можем нулевые значения медианным значением.

В двух столбцах есть пропущенные значения. Один из них — days_employed. Пропуски в этом столбце вы обработаете на следующем этапе.

* Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

In [6]:
# Замена нулевых значений в столбце 'total_income' медианным значением
# df['total_income'] = df['total_income'].fillna(df['total_income'].median())

# df.info() # Убеждаемся что нулевых значений больше нет

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в столбцах с аномалиями и опишите возможные причины появления таких данных.

Замените вещественный тип данных в столбце total_income на целочисленный, например, с помощью метода astype().

Если в данных присутствуют строки-дубликаты, удалите их. Также обработайте неявные дубликаты. Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к одному регистру. Проверьте остальные столбцы. После удаления дубликатов сделайте следующее:
* поясните, как выбирали метод для поиска и удаления дубликатов в данных;
* приведите возможные причины появления дубликатов.

Создайте два новых датафрейма со столбцами:
* education_id и education — в первом;
* family_status_id и family_status — во втором.

Удалите из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. Новые датафреймы — это те самые «словари» (не путайте с одноимённой структурой данных в Python), к которым вы сможете обращаться по идентификатору.

На основании диапазонов, указанных ниже, создайте столбец total_income_category с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Например, кредитополучателю с доходом 25000 нужно назначить категорию 'E', а клиенту, получающему 235000, — 'B'.

Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:
* `‘операции с автомобилем’`,
* `‘операции с недвижимостью’`,
* `‘проведение свадьбы’`,
* `‘получение образования’`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.
Вы можете использовать собственную функцию и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.

# 3. Ответы на вопросы

Ответы на вопросы можно разместить в ячейках тетрадок Jupyter Notebook с типом markdown.
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Ответы сопроводите интерпретацией — поясните, о чём именно говорит полученный вами результат.

# 4. Общий вывод